In [3]:
from PIL import Image                                              
import os, sys  
from pathlib import Path
import random

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import cv2
from sklearn.model_selection import train_test_split

In [4]:
path = (r"../input/exml-prelims-task-1")
dir_path = Path(path)
os.listdir(path)

In [5]:
test_dir_path = path + '/test/test'

In [6]:
train_dir = path + '/train/train'
os.listdir(train_dir)

In [7]:
dorsal_path = train_dir + '/dorsal'
dorsal_dir = Path(dorsal_path)
dorsal_filepaths = []
dorsal_list = []

for i in os.listdir(dorsal_path) :
    dorsal_filepaths.append(f"../input/exml-prelims-task-1/train/train/dorsal/{i}")

for i in range(0, len(dorsal_filepaths)):
    dorsal_list.append("dorsal")


print(f"Length of Dorsal Filepaths : {len(dorsal_filepaths)}\nLength of Dorsal List : {len(dorsal_list)}")

In [8]:
palmar_path = train_dir + '/palmar'
palmar_dir = Path(palmar_path)
palmar_filepaths = []
palmar_list = []

for i in os.listdir(palmar_path) :
    palmar_filepaths.append(f"../input/exml-prelims-task-1/train/train/palmar/{i}")

for i in range(0, len(palmar_filepaths)):
    palmar_list.append("palmar")


print(f"Length of Palmar Filepaths : {len(palmar_filepaths)}\nLength of Palmar List : {len(palmar_list)}")

In [9]:
all_filepaths = dorsal_filepaths + palmar_filepaths
labels = dorsal_list + palmar_list

train_df = {
    "Filepath" : all_filepaths,
    "Label" : labels
}

In [10]:
train_df = pd.DataFrame(train_df)
train_df

In [11]:
train_df, val_df = train_test_split(train_df, train_size=0.8, shuffle=True, random_state=1)

In [12]:
test_paths = []
sample_labels = []
for i in os.listdir(test_dir_path) :
    test_paths.append(f"../input/exml-prelims-task-1/test/test/{i}")
    
for i in range(0, len(test_paths)):
    sample_labels.append(random.choice(["Rick", "Astley"]))

test_dict = {
    "Filepath" : test_paths,
    "Label" : sample_labels
}    

test_df = pd.DataFrame(test_dict)
test_df

In [13]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    width_shift_range=0.2,
    height_shift_range=0.2,
    validation_split=0.2
)

val_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

In [14]:
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(160, 120),
    color_mode='rgb',
    class_mode='binary',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = val_generator.flow_from_dataframe(
    dataframe=val_df,
    x_col='Filepath',
    y_col = 'Label',
    target_size=(160, 120),
    color_mode='rgb',
    class_mode='binary',
    batch_size=16,
    shuffle=False
)

In [15]:
model = tf.keras.models.Sequential([

        tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(160, 120, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Dropout(0.3),

        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Dropout(0.3),

        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Dropout(0.3),

        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Dropout(0.3),

        tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),

        tf.keras.layers.Flatten(),
        
        tf.keras.layers.Dense(512, activation='relu'),
        
        tf.keras.layers.Dense(1, activation='sigmoid')])

In [16]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['binary_accuracy'])
model.summary()

In [17]:
history = model.fit(
    train_images,
    steps_per_epoch = (3200/32),
    epochs = 25,
    max_queue_size=100,
    workers = 4 ,
    use_multiprocessing=True,
    verbose = 1)

In [18]:
sub = pd.read_csv("../input/exml-prelims-task-1/SampleSubmission.csv")


folder = (r'../input/exml-prelims-task-1/test/test')
sub
# new_test_df = test_df.drop(['Label'], axis = 1)

In [19]:
a = []

for i in os.listdir(r'../input/exml-prelims-task-1/test/test'):
    a.append(i)

In [20]:
zeroes = []

for i in range(0, len(a)):
    zeroes.append(0)

In [21]:
sub_dict = {
    "id": a,
    "AspectofHand" : zeroes
}

In [22]:
len(zeroes)

In [23]:
raw_sub = pd.DataFrame(sub_dict)
raw_sub

In [24]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [25]:
test_generator =   datagen.flow_from_dataframe(raw_sub, folder, x_col = 'id', y_col=None,
    batch_size=32,
    class_mode=None,
    target_size =(160, 120),  
    shuffle=False,
    color_mode = 'rgb')

<a href="./model-sub2.hdf5"> Download File </a>

In [26]:
x = model.predict(test_generator)

In [27]:
sub['AspectofHand'] = x

In [28]:
sub[5:10]

In [29]:
model.save('model-sub2.hdf5')

In [30]:
sub.to_csv('submission.csv', index=False)